# Ценообразование опционов. Часть 5.

### Экзотические опционы и структурные продукты. Опцион на худший из активов с барьером

## Цели и задачи:
### 1. Изучить подходы к моделированию опционов и структурных продуктов на несколько активов (корзину активов)
### 2. Получить навык работы с классами Python
### 3. Получить дополнительные навыки работы с библиотеками NumPy и Pandas

 ### В данном ноутбуке рассматриваются подходы к ценообразованию струтурных продуктов. Полученные премии опционов могут оличичаться от фактических котировок, публикуемых  в соответствии с рыночной практикой.

### Рекомендуемая литература:
### 1.Levy G. Computational Finance. Numerical Methods for Pricing Financial Instruments
### 2.Neftci N.S. An Introduction to the Mathematics of Financial Derivatives
### 3.Hilpisch Y. Derivatives Analytics with Python: Data Analysis, Models, Simulation, Calibration and Hedging
### 4.London J. Modelling Derivatives with C++
### 5.Haug E.G. The Complete Guide to Option Pricing Formulas
### 6.De Weert F. Exotic Options Trading

## 1.Загрузка библиотек

In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math
from tqdm import tqdm # progress bar
from scipy import stats
import itertools
from scipy.linalg import cholesky

## 2.Классы, необходимые для реализации моделей ценообразования

### Броуновское движение - коррелированные случайные величины(шаг)

In [4]:
class GBM_step_correlated:
    def __init__(self,S,drift,dt,corr_matrix,sigma,div):
        self._S=S
        self._drift=drift
        self._corr_matrix=corr_matrix
        self._dt=dt
        self._sigma=sigma
        self._div=div
    
    def __random_normal_correlated__(self,mu=0,sigma=1):
        rvs=np.random.normal(mu,sigma,(self._corr_matrix.shape[0],1))
        c = cholesky(self._corr_matrix, lower=True)
        return np.dot(c,rvs)
            
    def get_step(self):
        return self._S*((self._drift-self._div)*self._dt+np.sqrt(self._dt)*sigma*self.__random_normal_correlated__())

### Класс для опциона worst-of with barrier ("опцион на худший из активов с барьером")

In [5]:
class Worst_of_with_barrier:
        def __init__(self,S,K,H,corr_matrix,sigma,div,r,drift,days,dt,rebate=0,nsim=10000):
            self._S=S
            self._H=H
            self._K=K
            self._r=r
            self._drift=drift
            self._div=div
            self._days=days
            self._corr_matrix=corr_matrix
            self._sigma=sigma
            self._dt=dt
            self._rebate=rebate
            self._nsim=nsim  
    
        def __generate_path__(self):
            path=[]
            S_current=self._S
            path.append(S_current)
            for _ in range(self._days): 
                step=GBM_step_correlated(S_current,self._drift,self._dt,self._corr_matrix,self._sigma,self._div)
                S_current=S_current+step.get_step() 
                #print(S_current)
                path.append(S_current)   
            return np.array(path)
            
        def __generate_payoff__(self): 
            payoff=0
            path=self.__generate_path__()
            if min(np.min(path,axis=0)/self._S)<self._H:
                payoff=max(0,self._K-min((path[-1,:]*np.exp(-self._r*days/250))/self._S))
               # print(path[-1,:])
              #  print(np.min(path,axis=0))
            return payoff
                 
        def __simulation__(self):
            payoffs=[]
            for _ in tqdm(range(self._nsim)):
                payoffs.append(self.__generate_payoff__())
            return payoffs 
        
        def get_price(self):
            payoffs=[]
            payoffs=self.__simulation__()
            return np.mean(payoffs)

## 3.Реализация опциона worst-of put with barrier

### Платежная функция опциона пут на худший из n активов опредедяется следующим образом:

### $Payoff=max[0,K-min(R_1(t),R_2(t),R_n(t))]$

### К этому необходимо добавить условия барьера. В общем случае задача не имеет аналитического решения, вследствие чего используется численное моделирование методом Монте-Карло.

### В качестве примера рассмотрим опцион пут с барьером на худший из трех активов: Intel, Oracle, Microsoft

### 3.1.Ковариационная и кореляционная матрицы

### Ковариационная матрица необходима для получения волатильности каждого актива

In [6]:
cov_matrix=pd.read_csv('/Users/massimozharkovsky/Documents/ML/cov_matrix')

In [7]:
cov_matrix

,ORCL,MSFT,IBM
0,0.000350,0.000259,0.000215
1,0.000259,0.000414,0.000239
2,0.000215,0.000239,0.000363


In [8]:
cov_matrix=cov_matrix.to_numpy()

In [9]:
cov_matrix

array([[0.0003497 , 0.00025914, 0.00021505],
       [0.00025914, 0.00041374, 0.00023863],
       [0.00021505, 0.00023863, 0.0003627 ]])

In [10]:
sigma=(np.diagonal(cov_matrix).reshape(cov_matrix.shape[0],1)*250)**.5

In [11]:
sigma

array([[0.29567856],
       [0.32161169],
       [0.30112291]])

### Кореляционая матрица используется в качестве входного параметра для моделирования коррелированных случайных величин

In [12]:
corr_matrix=pd.read_csv('/Users/massimozharkovsky/Documents/ML/cor_matrix')

In [13]:
corr_matrix=corr_matrix.to_numpy()

In [14]:
corr_matrix

array([[1.        , 0.68126557, 0.60382747],
       [0.68126557, 1.        , 0.61601169],
       [0.60382747, 0.61601169, 1.        ]])

### 3.2.Параметры опциона

In [15]:
S=np.array([124.4,72.16,227.4]).reshape(3,1)
div=np.array([0.01,0.01,0.01]).reshape(3,1)
H=.59 #барьер
K=1   #страйк 
dt=1/250
rebate=0
days=250
r=.01
drift=np.repeat(r,3).reshape(3,1)

In [16]:
drift

array([[0.01],
       [0.01],
       [0.01]])

In [17]:
wod=Worst_of_with_barrier(S,K,H,corr_matrix,sigma,div,r,drift,days,dt,rebate=rebate,nsim=10000)

In [18]:
wod.get_price()

100%|██████████| 10000/10000 [00:43<00:00, 232.47it/s]
/Users/massimozharkovsky/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


array([0.08725403])

### Для получения полной стоимости инструмента к полученной премии необходимо добавить ставку по депозиту (безрисковая ставка плюс премия за кредитный риск конкретного финансового института).